# Data Collection

## Importing Libraries

In [1]:
#import libraries
import requests
import json
import pandas as pd
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_rows', 400)
import os

#I will use this to put the data into a SQL database to be queried
from sqlalchemy import create_engine

## API Key

In [2]:
#import apikey
from api_key import api_key as key

# Data From EIA API

In [3]:
#creating a dictionary to store the ids for url requests
chart_dict = {}

In [4]:
#list of charts for the url requests
names = ['retail_gas_price',
         'regular_gas_price', 'premium_gas_price', 
         'europe_brent', 'wti', 'oil_supply', 'crude_oil_production']
#list of ids for the url requests
series_ids = ['TOTAL.MGUCUUS.M',  
              'PET.EMM_EPMR_PTE_NUS_DPG.M', 'PET.EMM_EPMP_PTE_NUS_DPG.M',
             'PET.RBRTE.M', 'PET.RWTC.M', 'PET.MTTUA_NUS_1.M', 'TOTAL.PAPRP48.M']

In [5]:
def get_chart(names, series_id):
    """
    Takes in a list of names and series_ids
    to create a dictionary of 
    chart names and urls
    """
    urls = []
    for i, n in enumerate(series_ids):
        url = 'http://api.eia.gov/series/?api_key={}&series_id={}'.format(key, series_id[i])
        urls.append(url)
    
    for i, n in enumerate(names):
        chart_dict[names[i]] = urls[i]
    return chart_dict

In [6]:
def make_call(url_value):
    """
    makes an API call
    INPUT: the url
    OUTPUT: the result of an API call
    """
    
    response = requests.get(url_value)
    
    return response.json()['series'][0]['data']

In [7]:
#saving the dictionary of chart names and urls to a variable
chart_dict = get_chart(names, series_ids)

In [8]:
def data_to_df(chart_dict):
    
    """
    This function takes in the dictionary of names and urls.
    From here it: Makes an API call, Sorts the data to be put in a
    DataFrame, & Creates the DataFrame(s)
    """
    
    data = make_call(chart_dict[names[count]])
    
    #sorts the data by date
    sorted_data = sorted(data, key = lambda x: x[0])

    #adding a dash to the DATE columns to allow it to be parsed
    for i, n in enumerate(sorted_data):
        sorted_data[i][0] = sorted_data[i][0][:4] + '/' + sorted_data[i][0][4:]
    
    #creating a dictionary to create a dataframe
    dataframe = {'date':[i[0] for i in sorted_data], names[count]:[j[1] for j in sorted_data]}

    #checks to see if the DataFrame exists
    path = os.path.exists('../Data/Prices.csv')
    
    if path == False:
        
        #creates an initial dataframe if there isn't one
        df1 = pd.DataFrame(dataframe)

        df1.to_csv('../Data/Prices.csv', index = False)

    else:
        #creates a new dataframe to be merged with the original
        df2 = pd.DataFrame(dataframe)
        
        df1 = pd.read_csv('../Data/Prices.csv')
        
        gas_data = pd.merge( df1, df2, how = 'left')
        
        gas_data.to_csv('../Data/Prices.csv', index = False) 
        

In [9]:
count = 0

#iterating through the charts to have 1 combined dataframe
while count != len(names):

    data_to_df(chart_dict)
    count +=1

In [10]:
gas = pd.read_csv('../Data/Prices.csv')
pd.set_option('display.max_columns', 40)

pd.set_option('display.max_rows', 400)

In [11]:
gas.head(2)

,date,retail_gas_price,regular_gas_price,premium_gas_price,europe_brent,wti,oil_supply,crude_oil_production
0,1973/01,NaN,NaN,NaN,NaN,NaN,NaN,8981.323
1,1973/02,NaN,NaN,NaN,NaN,NaN,NaN,9178.750


In [12]:
gas.dtypes

date                     object
retail_gas_price        float64
regular_gas_price       float64
premium_gas_price       float64
europe_brent            float64
wti                     float64
oil_supply              float64
crude_oil_production    float64
dtype: object

# Data From Federal Reserve Economic Data

In [13]:
def format_import_date(x):
    '''
    This function formats the dates in the imports csv
    to make it compatible with the other data.
    '''
    x=str(x)+'-01-01'
    return x

In [14]:
#getting the imports data
imports = pd.read_csv('../Data/additional_data/petroleum-consumption.csv', skiprows=4)
imports.rename(columns={'year': 'date'}, inplace = True)
imports = imports[['date', 'imports']]
imports['date'] = imports['date'].apply(format_import_date)
imports['date'] = pd.to_datetime(imports['date'])
imports['date'] = imports['date'].astype('object')
imports.to_csv('../Data/additional_data/imports.csv', index = False)

In [15]:
imports = pd.read_csv('additional_data/imports.csv')

In [16]:
imports.head()

,date,imports
0,1950-01-01 00:00:00,0.850
1,1951-01-01 00:00:00,0.844
2,1952-01-01 00:00:00,0.952
3,1953-01-01 00:00:00,1.034
4,1954-01-01 00:00:00,1.052


In [17]:
#list of the csv files for exogenous variables
csv_list = ['employees_oil_extraction', 'imports', 'inflation']

In [18]:
def add_new_data(csv_list, count):
    """
    This function takes in list of csvs collected from
    https://fred.stlouisfed.org/ & adds them to the main 
    dataset
    """
    df = pd.read_csv('../Data/additional_data/{}.csv'.format(csv_list[count]))
    
    #changing the column names to match the main dataset
    df.rename(columns={df.columns[0]:'date',df.columns[1]:csv_list[count]}, inplace = True)
    
    df['date'] = pd.to_datetime(df['date'])
    df['date'] = df['date'].astype('object')
    
    df.to_csv('../Data/additional_data/{}.csv'.format(csv_list[count]), index = False)
    
     #checks to see if the DataFrame exists
    path = os.path.exists('../Data/OutsideData.csv')
    
    if path == False:
        
        #creates an initial dataframe if there isn't one
        df1 = pd.read_csv('../Data/additional_data/{}.csv'.format(csv_list[count]))

        df1.to_csv('../Data/OutsideData.csv', index = False)

    else:
        #creates a new dataframe to be merged with the original
        df2 = pd.read_csv('../Data/additional_data/{}.csv'.format(csv_list[count]))
        
        df1 = pd.read_csv('../Data/OutsideData.csv')
        
        outside_data = pd.merge( df1, df2, how = 'left')
        
        outside_data.to_csv('../Data/OutsideData.csv', index = False) 

In [19]:
count = 0

#iterating through the csvs of exogenous variables to merge them with the dataframe
while count != len(csv_list):
    
    add_new_data(csv_list, count)
    count +=1

In [20]:
outside=pd.read_csv('../Data/OutsideData.csv')

outside.head()

,date,employees_oil_extraction,imports,inflation
0,1972-01-01 00:00:00,141.2,4.741,3.272278
1,1972-02-01 00:00:00,140.9,NaN,NaN
2,1972-03-01 00:00:00,140.8,NaN,NaN
3,1972-04-01 00:00:00,140.2,NaN,NaN
4,1972-05-01 00:00:00,139.6,NaN,NaN


# Data From Matteo Iacovello's GPR Dataset

In [21]:
def format_date(x):
    '''
    This function is for the GPR data. It formats the date 
    column so it can be parsed into a
    datetime object
    '''
    x = x.replace('-','/')
    if int(x[-2]) > 3:
        x = x[:4] + '19' + x[-2:]
    else:
        x = x[:4] + '20' + x[-2:]
    return x

In [22]:
#opening the GPR dataset
gpr = pd.read_csv('../Data/additional_data/gpr_countries.csv')
gpr.rename(columns={'Date':'date'}, inplace = True)
gpr['date'] = gpr['date'].apply(format_date)
gpr['date'] = pd.to_datetime(gpr['date'])
gpr['date'] = gpr['date'].astype('object')

outside = pd.read_csv('../Data/OutsideData.csv')

In [23]:
gpr['date'] = pd.to_datetime(gpr['date'])
outside['date'] = pd.to_datetime(outside['date'])

In [24]:
#concating the two to create the final dataset
outside_data = pd.merge(outside, gpr, how = 'left')

In [25]:
outside_data.tail()

,date,employees_oil_extraction,imports,inflation,GPR_TURKEY,GPR_MEXICO,GPR_KOREA,GPR_RUSSIA,GPR_INDIA,GPR_BRAZIL,GPR_CHINA,GPR_INDONESIA,GPR_SAUDI_ARABIA,GPR_SOUTH_AFRICA,GPR_ARGENTINA,GPR_COLOMBIA,GPR_VENEZUELA,GPR_THAILAND,GPR_UKRAINE,GPR_ISRAEL,GPR_MALAYSIA,GPR_PHILIPPINES,GPR_HONG_KONG
586,2020-11-01,139.4,NaN,NaN,105.830160,120.577633,92.987242,114.384703,80.411148,103.899544,136.882552,50.155582,65.022448,92.330456,58.675879,49.863902,98.778690,116.399298,198.549216,80.389557,46.136632,73.521926,203.471093
587,2020-12-01,139.5,NaN,NaN,85.124259,104.729149,117.483881,106.944362,83.112903,114.303653,138.807620,35.401636,97.054702,72.247458,66.643623,37.178948,87.733713,87.517346,165.840142,74.713905,108.410256,99.036877,202.440951
588,2021-01-01,133.0,NaN,NaN,117.276018,156.121912,109.809055,127.664855,84.002953,116.901498,141.333000,65.528941,100.736091,100.435826,70.527268,27.311767,88.869979,56.360588,259.116106,58.586234,59.089065,125.678506,201.224089
589,2021-02-01,132.7,NaN,NaN,73.636251,100.485698,76.973784,123.748078,69.875010,99.541834,126.667242,43.093446,90.078159,61.579600,49.148935,36.352542,69.595940,159.150604,226.745596,50.215048,53.826071,59.882763,179.267786
590,2021-03-01,133.3,NaN,NaN,82.737284,117.252892,142.811575,123.087580,86.535056,104.742674,137.439652,49.756466,95.671400,43.374098,52.635473,47.350546,81.098390,110.544503,178.320924,49.809270,84.451250,119.765526,185.763652


In [26]:
#saving the final data
outside_data.to_csv('../Data/OutsideData.csv')

In [27]:
prices = pd.read_csv('../Data/Prices.csv')

In [28]:
prices.dtypes

date                     object
retail_gas_price        float64
regular_gas_price       float64
premium_gas_price       float64
europe_brent            float64
wti                     float64
oil_supply              float64
crude_oil_production    float64
dtype: object

In [30]:
prices['date'] = pd.to_datetime(prices['date'])
prices.dtypes

date                    datetime64[ns]
retail_gas_price               float64
regular_gas_price              float64
premium_gas_price              float64
europe_brent                   float64
wti                            float64
oil_supply                     float64
crude_oil_production           float64
dtype: object

In [29]:
outside_data.dtypes

date                        datetime64[ns]
employees_oil_extraction           float64
imports                            float64
inflation                          float64
GPR_TURKEY                         float64
GPR_MEXICO                         float64
GPR_KOREA                          float64
GPR_RUSSIA                         float64
GPR_INDIA                          float64
GPR_BRAZIL                         float64
GPR_CHINA                          float64
GPR_INDONESIA                      float64
GPR_SAUDI_ARABIA                   float64
GPR_SOUTH_AFRICA                   float64
GPR_ARGENTINA                      float64
GPR_COLOMBIA                       float64
GPR_VENEZUELA                      float64
GPR_THAILAND                       float64
GPR_UKRAINE                        float64
GPR_ISRAEL                         float64
GPR_MALAYSIA                       float64
GPR_PHILIPPINES                    float64
GPR_HONG_KONG                      float64
dtype: obje